
# **Tutorial for Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines competition on drivendata.org**


The data for this tutorial can be obtained from this page https://www.drivendata.org/competitions/66/flu-shot-learning/ (you would have to register an account or just login with facebook or google+)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Downloading and importing dependencies**

In [2]:
!pip install catboost

     |████████████████████████████████| 64.8MB 63kB/s 


In [0]:
import numpy as np
import pandas as pd

# **Loading the datasets**

In [0]:
#Loading the train dataset containing only the training features
training_set_features = r"/content/drive/My Drive/LeadingIndiaAI/training_set_features.csv"
X = pd.read_csv(training_set_features,index_col="respondent_id")

#Loading the target label dataset
training_set_labels = r"/content/drive/My Drive/LeadingIndiaAI/training_set_labels.csv"
y = pd.read_csv(training_set_labels,index_col="respondent_id")

In [5]:
print("features_df.shape", X.shape)
X.head()

features_df.shape (26707, 35)


,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [6]:
X.dtypes

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                             object
income_poverty           

In [7]:
print("labels_df.shape", y.shape)
y.head()

labels_df.shape (26707, 2)


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [8]:
#Loading the test dataset
test_set_features = r"/content/drive/My Drive/LeadingIndiaAI/test_set_features.csv"
test_file = pd.read_csv(test_set_features,index_col="respondent_id")
test_file.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,1.0,1.0,18 - 34 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,4.0,2.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,4.0,4.0,4.0,2.0,35 - 44 Years,12 Years,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


# **Data Exploration**

In [9]:
#Checking the unique number of levels in all the categorical columns
for cat in ['age_group','education','race','sex', 'income_poverty', 'marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa','employment_industry','employment_occupation']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, X[cat].unique().size))

Number of levels in category 'age_group':  5.00 
Number of levels in category 'education':  5.00 
Number of levels in category 'race':  4.00 
Number of levels in category 'sex':  2.00 
Number of levels in category 'income_poverty':  4.00 
Number of levels in category 'marital_status':  3.00 
Number of levels in category 'rent_or_own':  3.00 
Number of levels in category 'employment_status':  4.00 
Number of levels in category 'hhs_geo_region':  10.00 
Number of levels in category 'census_msa':  3.00 
Number of levels in category 'employment_industry':  22.00 
Number of levels in category 'employment_occupation':  24.00 


In [10]:
#Checking the unique values in each level in all the categorical columns
for cat in ['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','census_msa','employment_industry','employment_occupation','hhs_geo_region']:
    print("Levels for catgeory '{0}': {1}".format(cat, X[cat].unique()))

Levels for catgeory 'age_group': ['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']
Levels for catgeory 'education': ['< 12 Years' '12 Years' 'College Graduate' 'Some College' nan]
Levels for catgeory 'race': ['White' 'Black' 'Other or Multiple' 'Hispanic']
Levels for catgeory 'sex': ['Female' 'Male']
Levels for catgeory 'income_poverty': ['Below Poverty' '<= $75,000, Above Poverty' '> $75,000' nan]
Levels for catgeory 'marital_status': ['Not Married' 'Married' nan]
Levels for catgeory 'rent_or_own': ['Own' 'Rent' nan]
Levels for catgeory 'employment_status': ['Not in Labor Force' 'Employed' 'Unemployed' nan]
Levels for catgeory 'census_msa': ['Non-MSA' 'MSA, Not Principle  City' 'MSA, Principle City']
Levels for catgeory 'employment_industry': [nan 'pxcmvdjn' 'rucpziij' 'wxleyezf' 'saaquncn' 'xicduogh' 'ldnlellj'
 'wlfvacwt' 'nduyfdeo' 'fcxhlnwr' 'vjjrobsf' 'arjwrbjb' 'atmlpfrs'
 'msuufmds' 'xqicxuve' 'phxvnwax' 'dotnnunm' 'mfikgejo' 'cfqqtusy'
 'mcubkhph' 

In [15]:
#Ensuring that there are no NaN values in the training dataset
X_new = X.fillna(-999)
pd.isnull(X_new).any()

h1n1_concern                   False
h1n1_knowledge                 False
behavioral_antiviral_meds      False
behavioral_avoidance           False
behavioral_face_mask           False
behavioral_wash_hands          False
behavioral_large_gatherings    False
behavioral_outside_home        False
behavioral_touch_face          False
doctor_recc_h1n1               False
doctor_recc_seasonal           False
chronic_med_condition          False
child_under_6_months           False
health_worker                  False
health_insurance               False
opinion_h1n1_vacc_effective    False
opinion_h1n1_risk              False
opinion_h1n1_sick_from_vacc    False
opinion_seas_vacc_effective    False
opinion_seas_risk              False
opinion_seas_sick_from_vacc    False
age_group                      False
education                      False
race                           False
sex                            False
income_poverty                 False
marital_status                 False
r

In [19]:
#Ensuring that there are no NaN values in the test dataset
test_file_new = test_file.fillna(-999)
pd.isnull(test_file_new).any()

h1n1_concern                   False
h1n1_knowledge                 False
behavioral_antiviral_meds      False
behavioral_avoidance           False
behavioral_face_mask           False
behavioral_wash_hands          False
behavioral_large_gatherings    False
behavioral_outside_home        False
behavioral_touch_face          False
doctor_recc_h1n1               False
doctor_recc_seasonal           False
chronic_med_condition          False
child_under_6_months           False
health_worker                  False
health_insurance               False
opinion_h1n1_vacc_effective    False
opinion_h1n1_risk              False
opinion_h1n1_sick_from_vacc    False
opinion_seas_vacc_effective    False
opinion_seas_risk              False
opinion_seas_sick_from_vacc    False
age_group                      False
education                      False
race                           False
sex                            False
income_poverty                 False
marital_status                 False
r

Define separate labels columns as target labels

In [0]:
#Our problem statement is Multi-label and catboost does'nt do this automatically at this point of time so we divide the target columns and repeat the 
#prediction process on them and later merge both the results into one submission dataframe
y_h1n1=y['h1n1_vaccine']
y_seas=y['seasonal_vaccine']

# **Build the model and fit it on the training dataset**

# Base approach

We were able to achieve 29th place on the leaderboard with this base approach, while doing no feature engineering at all: only few basic steps to prepare data and feed it into CatBoost model

Train the model and predict

In [20]:
#You can directly mention the categorical column labels or even indices for cat_features 
from catboost import CatBoostClassifier
cat_feat=['age_group','education','race','sex', 'income_poverty', 'marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa','employment_industry','employment_occupation']
model=CatBoostClassifier(loss_function='Logloss',cat_features=cat_feat)

#Fit the model on h1n1_vaccine target once and predict the probablity if the patient has taken the vaccine or not
model.fit(X_new,y_h1n1) 
pp1=model.predict_proba(test_file_new)

#Fit the model again on seasonal_vaccine target and predict the probablity if the patient has taken the vaccine or not
model.fit(X_new,y_seas) 
pp2=model.predict_proba(test_file_new)

Learning rate set to 0.041889
0:	learn: 0.6601656	total: 70.1ms	remaining: 1m 10s
1:	learn: 0.6306048	total: 143ms	remaining: 1m 11s
2:	learn: 0.6045621	total: 227ms	remaining: 1m 15s
3:	learn: 0.5800839	total: 296ms	remaining: 1m 13s
4:	learn: 0.5586265	total: 371ms	remaining: 1m 13s
5:	learn: 0.5389229	total: 443ms	remaining: 1m 13s
6:	learn: 0.5223090	total: 515ms	remaining: 1m 12s
7:	learn: 0.5072546	total: 590ms	remaining: 1m 13s
8:	learn: 0.4936809	total: 672ms	remaining: 1m 13s
9:	learn: 0.4813121	total: 741ms	remaining: 1m 13s
10:	learn: 0.4707138	total: 814ms	remaining: 1m 13s
11:	learn: 0.4601998	total: 896ms	remaining: 1m 13s
12:	learn: 0.4511600	total: 977ms	remaining: 1m 14s
13:	learn: 0.4438861	total: 1.05s	remaining: 1m 13s
14:	learn: 0.4367087	total: 1.13s	remaining: 1m 14s
15:	learn: 0.4301793	total: 1.2s	remaining: 1m 13s
16:	learn: 0.4242325	total: 1.27s	remaining: 1m 13s
17:	learn: 0.4197488	total: 1.32s	remaining: 1m 12s
18:	learn: 0.4142844	total: 1.39s	remaining:

In [21]:
print(pp1)

[[0.89154201 0.10845799]
 [0.97171616 0.02828384]
 [0.80540565 0.19459435]
 ...
 [0.8094761  0.1905239 ]
 [0.98124293 0.01875707]
 [0.49627294 0.50372706]]


In [22]:
print(pp2)

[[0.80709038 0.19290962]
 [0.9707959  0.0292041 ]
 [0.26235927 0.73764073]
 ...
 [0.80542638 0.19457362]
 [0.7609661  0.2390339 ]
 [0.35792562 0.64207438]]


In [0]:
#Load the sample submission file
sub_for = r"/content/drive/My Drive/LeadingIndiaAI/submission_format.csv"
submission_df = pd.read_csv(sub_for,index_col="respondent_id")

In [24]:
#Read both the target predictions and merge them
m=[]
for i in pp1:
  m.append(i[1])
print(m)

n=[]
for i in pp2:
  n.append(i[1])
print(n)
submission_df["h1n1_vaccine"] = m
submission_df["seasonal_vaccine"] = n

[0.10845798672385619, 0.02828383944464712, 0.19459434616256277, 0.6213672372030283, 0.3627268468675552, 0.8545041746717547, 0.29985127527934935, 0.17934908379556433, 0.023232330363762287, 0.23906007619380265, 0.00992107859653574, 0.3950852327994342, 0.06765539109737405, 0.019821310880623885, 0.36421666230066513, 0.10879527773157675, 0.8361356660795223, 0.01878929389105895, 0.007099439257694077, 0.04866588229627582, 0.5423819437344407, 0.24624709849860846, 0.21458967745021099, 0.04956066204962083, 0.030518805283022622, 0.1512060972392919, 0.9808531758159899, 0.7935478216526309, 0.08516550256709395, 0.07313714667777534, 0.07236630525204839, 0.8166477801065245, 0.15053586168794642, 0.1327541041803542, 0.048966637104091326, 0.10598137973085418, 0.06243264503921082, 0.034322889658513, 0.03604018160380086, 0.2981931960354789, 0.035948134211578456, 0.05405404792162942, 0.06389986916728094, 0.44651994362532427, 0.4897524564498764, 0.05900515443002706, 0.30799226633748794, 0.10265315286339632, 

Make submission

In [0]:
#Convert the dataframe into a csv file for submission
submission_df.to_csv(r'submission_final.csv', index=True)

# **Improved approach using Bagging**


Finally we are able to achieve 24th place on the leaderboard by averaging predictions from several models trained with different seed as we reduce the variance.

Train the model and predict

In [0]:
predictions_h1n1 = []

for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.1, iterations=1000, random_seed=i, logging_level='Silent')
    clf.fit(X_new, y_h1n1, cat_features=cat_feat)
    predictions_h1n1.append(clf.predict_proba(test_file_new)[:,1])


predictions_seas = []

for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.1, iterations=1000, random_seed=i, logging_level='Silent')
    clf.fit(X_new, y_seas, cat_features=cat_feat)
    predictions_seas.append(clf.predict_proba(test_file_new)[:,1])

Make submission

In [0]:
prediction_h1n1 = np.mean(predictions_h1n1, axis=0)
prediction_seas = np.mean(predictions_seas, axis=0)
pd.DataFrame(
    {'h1n1_vaccine':prediction_h1n1,'seasonal_vaccine':prediction_seas}
).to_csv(
    'submission_improved_bagged.csv', index=True
)